In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
#sys.path.insert(1, '/home/samanti/Desktop/THESIS_LMT/imps/src')

import numpy as np
import open3d as o3d
from tqdm.notebook import tqdm
from tqdm import tqdm
import torch
from matplotlib import cm
import matplotlib.pyplot as plt
import torch.optim as optim
import pandas as pd
from sklearn.metrics import jaccard_score

from imps.data.scannet import ScanNetScene, CLASS_NAMES
from imps.sqn.model import Randla
from imps.sqn.data_utils import prepare_input

from imps.point_augment.Common import loss_utils

from imps.point_augment.Augmentor.augmentor import Augmentor
from imps.point_augment.Common import loss_utils, point_augment_utils

SCENE_DIR = '/app/mnt/scans_test'

N_POINTS = int(1.5e5)
RESOLUTION = 25
SIGMAS = None
DEVICE = 'cpu'

IGNORED_LABELS = [0]


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_iou(logits, labels):
    preds = np.argmax(logits, axis=-1)
    ious = []
    
    for c in range(len(CLASS_NAMES)):
        iou = jaccard_score((labels==c).astype(int), (preds==c).astype(int), pos_label=1)
        ious.append(iou)
        
    return np.array(ious)

In [3]:
SCENE_DIR_TRAIN ='/app/mnt/scans'

In [4]:
with open('../../data/train_dataset.txt') as f:
    train_data = f.read().splitlines() 

In [5]:
scene_names_train = os.listdir(SCENE_DIR_TRAIN)

In [6]:
extra_scenes = list(set(scene_names_train).difference(train_data))

In [7]:
len(extra_scenes)

314

In [12]:
test_scenes = extra_scenes[0:101]

In [13]:
index_list = []
for item in test_scenes:    
    if item.endswith(".zip"):        
        index = test_scenes.index(item)
        index_list.append(index)
        
for index in sorted(index_list, reverse=True):
    del test_scenes[index]
        

In [14]:
print(len(test_scenes))

100


In [101]:
test_scenes_2 = extra_scenes[102:203]

In [105]:
len(test_scenes_2)

100

In [4]:
test_scenes = ['scene0664_02', 'scene0583_02', 'scene0316_00', 'scene0702_01', 'scene0300_01', 'scene0050_00', 'scene0599_01', 'scene0559_00', 'scene0064_01', 'scene0334_02', 'scene0701_02', 'scene0664_01', 'scene0558_00', 'scene0695_01', 'scene0046_00', 'scene0652_00', 'scene0256_02', 'scene0187_00', 'scene0629_00', 'scene0678_02', 'scene0684_01', 'scene0655_00', 'scene0690_01', 'scene0217_00', 'scene0435_02', 'scene0406_02', 'scene0423_01', 'scene0591_01', 'scene0647_00', 'scene0086_02', 'scene0328_00', 'scene0406_01', 'scene0629_01', 'scene0686_02', 'scene0050_02', 'scene0474_01', 'scene0046_02', 'scene0081_02', 'scene0025_01', 'scene0690_00', 'scene0685_02', 'scene0527_00', 'scene0653_00', 'scene0011_01', 'scene0686_00', 'scene0084_00', 'scene0671_01', 'scene0653_01', 'scene0575_02', 'scene0222_00', 'scene0696_02', 'scene0257_00', 'scene0164_00', 'scene0643_00', 'scene0357_00', 'scene0278_01', 'scene0695_03', 'scene0500_00', 'scene0338_02', 'scene0019_01', 'scene0077_01', 'scene0084_01', 'scene0088_01', 'scene0518_00', 'scene0378_02', 'scene0353_01', 'scene0684_00', 'scene0458_00', 'scene0658_00', 'scene0693_01', 'scene0063_00', 'scene0207_01', 'scene0427_00', 'scene0146_00', 'scene0382_01', 'scene0149_00', 'scene0153_01', 'scene0633_01', 'scene0609_01', 'scene0553_00', 'scene0343_00', 'scene0329_00', 'scene0378_00', 'scene0458_01', 'scene0648_00', 'scene0568_01', 'scene0354_00', 'scene0580_00', 'scene0648_01', 'scene0187_01', 'scene0356_00', 'scene0651_02', 'scene0609_02', 'scene0591_00', 'scene0644_00', 'scene0432_01', 'scene0131_00', 'scene0426_02', 'scene0701_01', 'scene0169_00']

In [102]:
print(test_scenes_2)

['scene0552_00', 'scene0406_00', 'scene0412_00', 'scene0700_02', 'scene0559_01', 'scene0100_02', 'scene0700_00', 'scene0329_01', 'scene0011_00', 'scene0704_01', 'scene0256_00', 'scene0686_01', 'scene0351_00', 'scene0329_02', 'scene0693_00', 'scene0064_00', 'scene0598_00', 'scene0355_01', 'scene0574_01', 'scene0607_00', 'scene0616_01', 'scene0496_00', 'scene0164_03', 'scene0462_00', 'scene0196_00', 'scene0608_01', 'scene0088_00', 'scene0353_00', 'scene0678_01', 'scene0430_00', 'scene0100_01', 'scene0488_01', 'scene0000_00.zip', 'scene0598_02', 'scene0552_01', 'scene0231_02', 'scene0307_00', 'scene0356_02', 'scene0356_01', 'scene0474_03', 'scene0086_01', 'scene0608_02', 'scene0389_00', 'scene0251_00', 'scene0414_00', 'scene0593_00', 'scene0578_01', 'scene0207_02', 'scene0435_00', 'scene0277_00', 'scene0307_02', 'scene0494_00', 'scene0645_00', 'scene0580_01', 'scene0704_00', 'scene0618_00', 'scene0334_00', 'scene0377_00', 'scene0131_01', 'scene0025_02', 'scene0256_01', 'scene0030_01', 'sc

In [103]:
index_list = []
for item in test_scenes_2:    
    if item.endswith(".zip"):        
        index = test_scenes_2.index(item)
        index_list.append(index)
        
for index in sorted(index_list, reverse=True):
    del test_scenes_2[index]
        

In [5]:
len(test_scenes)

100

In [5]:
randla = Randla(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], device=DEVICE, num_class=len(CLASS_NAMES), interpolator='keops')
randla.load_state_dict(torch.load('../../processed/saved_models/10_epochs_1201_scenes_PA_extension_4_final_with_disp_and_flip_at_center'))
randla.eval()

Randla(
  (fc0): Conv1d(
    (conv): Conv1d(3, 8, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(
      (bn): BatchNorm1d(8, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
    )
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (dilated_res_blocks): ModuleList(
    (0): Dilated_res_block(
      (mlp1): Conv2d(
        (conv): Conv2d(8, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(
          (bn): BatchNorm2d(4, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
        )
        (activation): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (lfa): Building_block(
        (mlp1): Conv2d(
          (conv): Conv2d(10, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(
            (bn): BatchNorm2d(4, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
          )
          (activation): LeakyReLU(negative_slope=0.2, inplace=True)
        )
  

In [6]:
mIOU_list = []
for sn in tqdm(test_scenes):
    scene = ScanNetScene(os.path.join(SCENE_DIR_TRAIN, sn))
    voxel_grid, surface_points, surface_colors, vicinity_points, vicinity_distances, point_labels = scene.create_if_data(RESOLUTION, N_POINTS, SIGMAS)
    surface_points_, center, pts_min, pts_max = point_augment_utils.scaling_point_cloud(surface_points)
    class_counts = []
    for c in range(len(CLASS_NAMES.keys())):
        class_counts.append(np.sum(point_labels == c))
    class_counts = np.array(class_counts)

    for ign in IGNORED_LABELS:
        class_counts[ign] = 0
    class_weights = class_counts / class_counts.sum()
    features = torch.FloatTensor(surface_colors).unsqueeze(0).to(DEVICE)
    xyz = torch.FloatTensor(surface_points_).unsqueeze(0)
    point_labels = torch.LongTensor(point_labels).unsqueeze(0).to(DEVICE)

    input_points, input_neighbors, input_pools, feat_shape = prepare_input(xyz, k=16, num_layers=3, encoder_dims = [8,32,64], sub_sampling_ratio=4, 
                                                               device=DEVICE)
    
    with torch.no_grad():
        all_logits = randla.forward(features, input_points, input_neighbors, input_pools)
        all_logits = all_logits.squeeze().detach().numpy()
        
    logits_randla_augment = scene.colorize_labels(all_logits.argmax(axis=-1))
    with open(f'../../processed/logits_folder/final_logits_randla_PA_{sn}.npy', 'wb') as f:
        np.save(f, logits_randla_augment)
    with open(f'../../processed/surface_points_folder/surface_points_randla_PA_{sn}.npy', 'wb') as f:
        np.save(f, surface_points)
    df = pd.DataFrame()
    df['class'] = CLASS_NAMES.keys()
    df['iou'] = get_iou(all_logits, point_labels.detach().squeeze().numpy())
    df['weight'] = class_weights
    df['iou_weighted'] = df['iou'] * df['weight']
    df.to_csv(f'../../processed/csv_iou_of_instances/{sn}_dataframe.csv', index=False)
    print(df)

    mIOU = df.iou_weighted.sum()
    print("mIOU given by: {}".format(mIOU))
    mIOU_list.append(mIOU)
    print(mIOU_list)

  0%|                                                                                                                                                                                       | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_d

             class       iou    weight  iou_weighted
0      unannotated  0.000000  0.000000      0.000000
1             wall  0.239895  0.308679      0.074051
2            floor  0.572422  0.186299      0.106642
3          cabinet  0.000000  0.000000      0.000000
4              bed  0.000000  0.000000      0.000000
5            chair  0.000000  0.000000      0.000000
6             sofa  0.000000  0.000000      0.000000
7            table  0.000000  0.000000      0.000000
8             door  0.000000  0.139980      0.000000
9           window  0.000000  0.000000      0.000000
10       bookshelf  0.000000  0.000000      0.000000
11         picture  0.000000  0.024425      0.000000
12         counter  0.000000  0.071000      0.000000
13            desk  0.000000  0.000000      0.000000
14         curtain  0.000000  0.000000      0.000000
15    refrigerator  0.000000  0.000000      0.000000
16   showercurtain  0.000000  0.000000      0.000000
17          toilet  0.000000  0.061956      0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

             class       iou    weight  iou_weighted
0      unannotated  0.000000  0.000000      0.000000
1             wall  0.326538  0.419808      0.137083
2            floor  0.780935  0.236196      0.184453
3          cabinet  0.000000  0.000000      0.000000
4              bed  0.000000  0.000000      0.000000
5            chair  0.000000  0.000000      0.000000
6             sofa  0.000000  0.000000      0.000000
7            table  0.000000  0.000000      0.000000
8             door  0.000000  0.102934      0.000000
9           window  0.000000  0.081850      0.000000
10       bookshelf  0.000000  0.000000      0.000000
11         picture  0.000000  0.022804      0.000000
12         counter  0.000000  0.000000      0.000000
13            desk  0.000000  0.000000      0.000000
14         curtain  0.000000  0.000000      0.000000
15    refrigerator  0.000000  0.000000      0.000000
16   showercurtain  0.000000  0.000000      0.000000
17          toilet  0.000000  0.036522      0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

...

             class       iou    weight  iou_weighted
0      unannotated  0.000000  0.000000      0.000000
1             wall  0.422215  0.491829      0.207657
2            floor  0.712201  0.192956      0.137423
3          cabinet  0.000000  0.000000      0.000000
4              bed  0.000000  0.086530      0.000000
5            chair  0.053575  0.052505      0.002813
6             sofa  0.000000  0.000000      0.000000
7            table  0.000000  0.000000      0.000000
8             door  0.000000  0.072722      0.000000
9           window  0.000000  0.049216      0.000000
10       bookshelf  0.000000  0.000000      0.000000
11         picture  0.000000  0.008581      0.000000
12         counter  0.000000  0.000000      0.000000
13            desk  0.000000  0.030958      0.000000
14         curtain  0.000000  0.000000      0.000000
15    refrigerator  0.000000  0.000000      0.000000
16   showercurtain  0.000000  0.000000      0.000000
17          toilet  0.000000  0.000000      0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

             class       iou    weight  iou_weighted
0      unannotated  0.000000  0.000000      0.000000
1             wall  0.226993  0.125299      0.028442
2            floor  0.928456  0.252052      0.234020
3          cabinet  0.000000  0.000000      0.000000
4              bed  0.000000  0.000000      0.000000
5            chair  0.000000  0.000000      0.000000
6             sofa  0.000000  0.538892      0.000000
7            table  0.000000  0.000000      0.000000
8             door  0.000000  0.000000      0.000000
9           window  0.000000  0.000000      0.000000
10       bookshelf  0.000000  0.000000      0.000000
11         picture  0.000000  0.000000      0.000000
12         counter  0.000000  0.000000      0.000000
13            desk  0.000000  0.000000      0.000000
14         curtain  0.000000  0.000000      0.000000
15    refrigerator  0.000000  0.000000      0.000000
16   showercurtain  0.000000  0.000000      0.000000
17          toilet  0.000000  0.000000      0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

             class       iou    weight  iou_weighted
0      unannotated  0.000000  0.000000      0.000000
1             wall  0.476160  0.279766      0.133214
2            floor  0.905091  0.343277      0.310697
3          cabinet  0.000000  0.056749      0.000000
4              bed  0.000000  0.000000      0.000000
5            chair  0.425240  0.169884      0.072241
6             sofa  0.000000  0.000000      0.000000
7            table  0.000000  0.084409      0.000000
8             door  0.000000  0.022411      0.000000
9           window  0.000000  0.023001      0.000000
10       bookshelf  0.000000  0.000000      0.000000
11         picture  0.000000  0.009704      0.000000
12         counter  0.000000  0.000000      0.000000
13            desk  0.000000  0.000000      0.000000
14         curtain  0.000000  0.000000      0.000000
15    refrigerator  0.000000  0.000000      0.000000
16   showercurtain  0.000000  0.000000      0.000000
17          toilet  0.000000  0.000000      0.

In [7]:
from statistics import mean
mean_IOU_performance = mean(mIOU_list)

In [ ]:
from statistics import mean
mean(mIOU_list)
max(mIOU_list)
min(mIOU_list)
mIOU_list_RandLA_with_PA = mIOU_list
list_of_better_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if i > j ]
list_of_worse_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if j -i > 0.1 ]
mIOU_list_RandLA_with_PA.index(0.6627236761349584)
mIOU_list_RandLA[16]
test_scenes[16]
print(mIOU_list_RandLA)
print(mIOU_list_RandLA_with_PA)


In [8]:
max(mIOU_list)

0.7261748689212824

In [9]:
mean_IOU_performance

0.3973365987442961

In [10]:
min(mIOU_list)

0.17547518759879904

mIOU_list_RandLA = [0.2962460688548717, 0.30391794630616453, 0.6524141724191902, 0.5080124211021342, 0.2546997191227881, 0.3242468249175954, 0.48479004956123334, 0.5467472828283928, 0.25181696633670264, 0.3229743586259389, 0.1806640666147666, 0.31367196910536943, 0.3036725050839316, 0.3747066545714159, 0.3572061277044272, 0.40287405829159245, 0.36737397224146057, 0.6353939891926504, 0.3761727240153292, 0.6351422203946238, 0.560441796027732, 0.5107469284348097, 0.20716565277126062, 0.4451154906483967, 0.4253084700905224, 0.2904131535243195, 0.5909509165572139, 0.31562296796897144, 0.5209770018455523, 0.4975059521240728, 0.21717606360205344, 0.4317293892832681, 0.3238602108257816, 0.6895299081971155, 0.3690091317136916, 0.2874771060219107, 0.2929327783651647, 0.5474389781053786, 0.40384188640793345, 0.2342361781598103, 0.5722761486238237, 0.30507412415321594, 0.4203328130750395, 0.4809527584990411, 0.6434470726429078, 0.5296326167966681, 0.6076567618895583, 0.3910390506826292, 0.609951648290316, 0.2927773618419275, 0.18319431809882947, 0.49713841561146815, 0.34176421620261693, 0.37245366506176564, 0.19209016490995237, 0.33005224730456395, 0.3384947301116933, 0.40968403591397695, 0.4928449811285595, 0.4932067019612236, 0.30958784523423216, 0.49959529400325586, 0.5886512915627848, 0.2572291289098335, 0.24310835573672146, 0.3188433077696804, 0.5630492219991903, 0.23276077591441402, 0.4137847487931866, 0.3489074211121302, 0.6493388201758812, 0.3443108298594106, 0.5794664631258206, 0.6982417775139159, 0.4302399721903458, 0.3453419102224347, 0.41945000398349075, 0.1987362830933742, 0.747812781639543, 0.6315941225159131, 0.5071202797293516, 0.5021270189804147, 0.21253000147586426, 0.18587621726507805, 0.2636519379172556, 0.44476835090634154, 0.630801500681901, 0.2448259859103279, 0.27981764866435677, 0.5568529939105378, 0.4689456178729721, 0.4991825337635197, 0.49273473338235046, 0.1906291327494808, 0.36953248943600503, 0.5751055744192958, 0.5104781172111208, 0.3942255688856783, 0.24812665336476292, 0.5404031854064268]

In [11]:
mIOU_list_RandLA = [0.2962460688548717, 0.30391794630616453, 0.6524141724191902, 0.5080124211021342, 0.2546997191227881, 0.3242468249175954, 0.48479004956123334, 0.5467472828283928, 0.25181696633670264, 0.3229743586259389, 0.1806640666147666, 0.31367196910536943, 0.3036725050839316, 0.3747066545714159, 0.3572061277044272, 0.40287405829159245, 0.36737397224146057, 0.6353939891926504, 0.3761727240153292, 0.6351422203946238, 0.560441796027732, 0.5107469284348097, 0.20716565277126062, 0.4451154906483967, 0.4253084700905224, 0.2904131535243195, 0.5909509165572139, 0.31562296796897144, 0.5209770018455523, 0.4975059521240728, 0.21717606360205344, 0.4317293892832681, 0.3238602108257816, 0.6895299081971155, 0.3690091317136916, 0.2874771060219107, 0.2929327783651647, 0.5474389781053786, 0.40384188640793345, 0.2342361781598103, 0.5722761486238237, 0.30507412415321594, 0.4203328130750395, 0.4809527584990411, 0.6434470726429078, 0.5296326167966681, 0.6076567618895583, 0.3910390506826292, 0.609951648290316, 0.2927773618419275, 0.18319431809882947, 0.49713841561146815, 0.34176421620261693, 0.37245366506176564, 0.19209016490995237, 0.33005224730456395, 0.3384947301116933, 0.40968403591397695, 0.4928449811285595, 0.4932067019612236, 0.30958784523423216, 0.49959529400325586, 0.5886512915627848, 0.2572291289098335, 0.24310835573672146, 0.3188433077696804, 0.5630492219991903, 0.23276077591441402, 0.4137847487931866, 0.3489074211121302, 0.6493388201758812, 0.3443108298594106, 0.5794664631258206, 0.6982417775139159, 0.4302399721903458, 0.3453419102224347, 0.41945000398349075, 0.1987362830933742, 0.747812781639543, 0.6315941225159131, 0.5071202797293516, 0.5021270189804147, 0.21253000147586426, 0.18587621726507805, 0.2636519379172556, 0.44476835090634154, 0.630801500681901, 0.2448259859103279, 0.27981764866435677, 0.5568529939105378, 0.4689456178729721, 0.4991825337635197, 0.49273473338235046, 0.1906291327494808, 0.36953248943600503, 0.5751055744192958, 0.5104781172111208, 0.3942255688856783, 0.24812665336476292, 0.5404031854064268]

In [ ]:
mIOU_list_RandLA_with_PA_ext_4 = [0.18879058175445823, 0.3347260513507414, 0.6478294183072476, 0.4634004828877911, 0.2776883835204738, 0.3348885181810214, 0.5303280696739987, 0.5112778797655251, 0.2826649605720013, 0.3600127043617918, 0.22264504339512847, 0.21348816109170277, 0.3533759439554946, 0.2860020938151539, 0.3099882570574908, 0.3601701037167576, 0.22957549777758407, 0.6822172081342764, 0.4505252575745559, 0.6261309063627546, 0.5794309483603944, 0.596951909061554, 0.2084514659521479, 0.33751871821287965, 0.29645764266278674, 0.20897083174719921, 0.6720113481448065, 0.2918544167564496, 0.5185960778074885, 0.4973000211567318, 0.3181248418589943, 0.3901624865483329, 0.36830282954774735, 0.681017812332608, 0.3535064713591302, 0.27980628516450595, 0.26011590533704154, 0.6843788026624392, 0.3842073486896482, 0.2613142325852199, 0.5510211921914989, 0.32977787395142233, 0.38017491255157876, 0.5427181287023839, 0.6332688689986895, 0.5006350338353477, 0.6754017376944329, 0.38583067146353167, 0.5790966451229402, 0.23163415333407722, 0.25248308709591005, 0.5268382635400548, 0.3240680055000821, 0.3707717777146439, 0.22030462774308054, 0.2905302385644058, 0.30131927983365925, 0.46511086952622704, 0.4505421611303456, 0.4648866713926567, 0.331973447476165, 0.47399394536847295, 0.4962096538006703, 0.27543395164334616, 0.23125023642545783, 0.2676173053710139, 0.6000843549930012, 0.22692104921053646, 0.37965499480609355, 0.3210738206835634, 0.6384296507456603, 0.3423464610308608, 0.5947581707948719, 0.6726786170392266, 0.4115534126543949, 0.3480345862620991, 0.3666430424870848, 0.18583524922899758, 0.814114558554003, 0.6171759591519677, 0.5951808350157984, 0.5142536428112097, 0.20388409320638426, 0.1904466835002704, 0.18994485290235325, 0.4398273987857711, 0.6245750640949566, 0.2045112973670595, 0.20915419496518206, 0.6629223340828474, 0.4149522045203412, 0.4928281919638059, 0.6378984233547755, 0.19094333383914594, 0.4714270982443618, 0.636694675574303, 0.459842786727518, 0.3392359496550571, 0.2663764527270179, 0.5049513794552815]

In [12]:
mIOU_list_RandLA_with_PA_ext_4_disp_FAC = mIOU_list

In [13]:
list_of_better_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA_ext_4_disp_FAC, mIOU_list_RandLA) if i > j ]

In [91]:
list_of_worse_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if j -i > 0.1 ]

In [92]:
list_of_worse_performing_scenes

[0.16815843350309098,
 0.1964683822255132,
 0.22428943125891432,
 0.3352736830460327,
 0.3093818418577592,
 0.18664869539463694,
 0.23102064956346108]

In [14]:
len(list_of_better_performing_scenes)

33

In [16]:
print(mIOU_list_RandLA_with_PA)

[0.18879058175445823, 0.3347260513507414, 0.6478294183072476, 0.4634004828877911, 0.2776883835204738, 0.3348885181810214, 0.5303280696739987, 0.5112778797655251, 0.2826649605720013, 0.3600127043617918, 0.22264504339512847, 0.21348816109170277, 0.3533759439554946, 0.2860020938151539, 0.3099882570574908, 0.3601701037167576, 0.22957549777758407, 0.6822172081342764, 0.4505252575745559, 0.6261309063627546, 0.5794309483603944, 0.596951909061554, 0.2084514659521479, 0.33751871821287965, 0.29645764266278674, 0.20897083174719921, 0.6720113481448065, 0.2918544167564496, 0.5185960778074885, 0.4973000211567318, 0.3181248418589943, 0.3901624865483329, 0.36830282954774735, 0.681017812332608, 0.3535064713591302, 0.27980628516450595, 0.26011590533704154, 0.6843788026624392, 0.3842073486896482, 0.2613142325852199, 0.5510211921914989, 0.32977787395142233, 0.38017491255157876, 0.5427181287023839, 0.6332688689986895, 0.5006350338353477, 0.6754017376944329, 0.38583067146353167, 0.5790966451229402, 0.231634

In [ ]:
mIOU_list_RandLA_with_PA_ext_4 = [0.18879058175445823, 0.3347260513507414, 0.6478294183072476, 0.4634004828877911, 0.2776883835204738, 0.3348885181810214, 0.5303280696739987, 0.5112778797655251, 0.2826649605720013, 0.3600127043617918, 0.22264504339512847, 0.21348816109170277, 0.3533759439554946, 0.2860020938151539, 0.3099882570574908, 0.3601701037167576, 0.22957549777758407, 0.6822172081342764, 0.4505252575745559, 0.6261309063627546, 0.5794309483603944, 0.596951909061554, 0.2084514659521479, 0.33751871821287965, 0.29645764266278674, 0.20897083174719921, 0.6720113481448065, 0.2918544167564496, 0.5185960778074885, 0.4973000211567318, 0.3181248418589943, 0.3901624865483329, 0.36830282954774735, 0.681017812332608, 0.3535064713591302, 0.27980628516450595, 0.26011590533704154, 0.6843788026624392, 0.3842073486896482, 0.2613142325852199, 0.5510211921914989, 0.32977787395142233, 0.38017491255157876, 0.5427181287023839, 0.6332688689986895, 0.5006350338353477, 0.6754017376944329, 0.38583067146353167, 0.5790966451229402, 0.23163415333407722, 0.25248308709591005, 0.5268382635400548, 0.3240680055000821, 0.3707717777146439, 0.22030462774308054, 0.2905302385644058, 0.30131927983365925, 0.46511086952622704, 0.4505421611303456, 0.4648866713926567, 0.331973447476165, 0.47399394536847295, 0.4962096538006703, 0.27543395164334616, 0.23125023642545783, 0.2676173053710139, 0.6000843549930012, 0.22692104921053646, 0.37965499480609355, 0.3210738206835634, 0.6384296507456603, 0.3423464610308608, 0.5947581707948719, 0.6726786170392266, 0.4115534126543949, 0.3480345862620991, 0.3666430424870848, 0.18583524922899758, 0.814114558554003, 0.6171759591519677, 0.5951808350157984, 0.5142536428112097, 0.20388409320638426, 0.1904466835002704, 0.18994485290235325, 0.4398273987857711, 0.6245750640949566, 0.2045112973670595, 0.20915419496518206, 0.6629223340828474, 0.4149522045203412, 0.4928281919638059, 0.6378984233547755, 0.19094333383914594, 0.4714270982443618, 0.636694675574303, 0.459842786727518, 0.3392359496550571, 0.2663764527270179, 0.5049513794552815]

In [223]:
mIOU_list_RandLA_with_PA.index(0.4881896270296582)

45

In [152]:
mIOU_list_RandLA[92]

0.569316053154611

In [224]:
test_scenes[45]

'scene0084_00'

##### print(mIOU_list_RandLA)

In [172]:
mIOU_list_RandLA_with_PA=[0.16815843350309098, 0.3415562553967516, 0.6413073757715071, 0.44933173185155156, 0.2577113296366025, 0.34256285707121126, 0.5017595225160139, 0.5009495466581202, 0.2879120399442017, 0.36757050129530205, 0.21386428723922793, 0.1964683822255132, 0.3019710384781971, 0.31034589881267693, 0.30310950917449897, 0.3214969048316504, 0.22428943125891432, 0.6445168556879105, 0.4658184734607898, 0.6229733528842784, 0.5455498814965294, 0.4892850251366492, 0.21457413633381167, 0.3352736830460327, 0.3093818418577592, 0.18664869539463694, 0.7599699837947658, 0.2964842933574372, 0.5185382495886818, 0.48015463668166125, 0.31210994690396604, 0.39775706132984845, 0.405679453498883, 0.6461731922059093, 0.3638125360009157, 0.2859883735945055, 0.24382862082068968, 0.6246785782230996, 0.38104765757533005, 0.26906794355704017, 0.6066627522419569, 0.29883084660317694, 0.3884125473484074, 0.4935916285284703, 0.6036383831088439, 0.4881896270296582, 0.6268076476281406, 0.3847181956333663, 0.5801406716559462, 0.24098854625832583, 0.216407589070414, 0.5228084950013853, 0.32696965060806327, 0.3465286317967129, 0.21367594941377283, 0.28214048375035855, 0.3488285150738589, 0.34870118923519877, 0.48985481243448215, 0.49589567948364555, 0.3352696061596157, 0.44998729716333785, 0.5129950528088635, 0.25219308711134547, 0.23159249048802713, 0.2920773492917436, 0.5495730951640604, 0.21549199913076852, 0.39118004101166287, 0.23102064956346108, 0.6768431059019315, 0.3426620004041056, 0.614984557167862, 0.6791208623521419, 0.42478168087189183, 0.383009154633105, 0.3417973919470071, 0.17525045296361308, 0.7479313172676266, 0.5814990610292321, 0.46954875846586364, 0.5203534354037826, 0.19473139344118667, 0.19602559698266048, 0.21307337700258805, 0.4370624735736105, 0.5346371305518502, 0.2100901811446143, 0.23093257853369317, 0.6420387870698748, 0.4026483320227573, 0.4645695570563021, 0.6152977869456445, 0.18388222093722592, 0.37288944257190393, 0.6627236761349584, 0.4638184010325454, 0.3584454329960413, 0.2696416754474739, 0.49070737631382927]


In [32]:
list_of_better_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if i > j ]

[0.3415562553967516,
 0.2577113296366025,
 0.34256285707121126,
 0.5017595225160139,
 0.2879120399442017,
 0.36757050129530205,
 0.21386428723922793,
 0.6445168556879105,
 0.4658184734607898,
 0.21457413633381167,
 0.7599699837947658,
 0.31210994690396604,
 0.405679453498883,
 0.6246785782230996,
 0.26906794355704017,
 0.6066627522419569,
 0.4935916285284703,
 0.6268076476281406,
 0.216407589070414,
 0.5228084950013853,
 0.21367594941377283,
 0.3488285150738589,
 0.49589567948364555,
 0.3352696061596157,
 0.6768431059019315,
 0.614984557167862,
 0.383009154633105,
 0.7479313172676266,
 0.5203534354037826,
 0.19602559698266048,
 0.6420387870698748,
 0.6152977869456445,
 0.37288944257190393,
 0.6627236761349584,
 0.2696416754474739]

In [29]:
mIOU_list_RandLA_with_PA = mIOU_list

In [ ]:
import seaborn as sns
ax = sns.boxplot(x = mIOU_list)
ax.get_figure().savefig('../../processed/figures/PA_Extension_4_entire_dataset.png')

In [106]:
mIOU_list = []
for sn in tqdm(test_scenes_2):
    scene = ScanNetScene(os.path.join(SCENE_DIR_TRAIN, sn))
    voxel_grid, surface_points, surface_colors, vicinity_points, vicinity_distances, point_labels = scene.create_if_data(RESOLUTION, N_POINTS, SIGMAS)
    surface_points_, center, pts_min, pts_max = point_augment_utils.scaling_point_cloud(surface_points)
    class_counts = []
    for c in range(len(CLASS_NAMES.keys())):
        class_counts.append(np.sum(point_labels == c))
    class_counts = np.array(class_counts)

    for ign in IGNORED_LABELS:
        class_counts[ign] = 0
    class_weights = class_counts / class_counts.sum()
    features = torch.FloatTensor(surface_colors).unsqueeze(0).to(DEVICE)
    xyz = torch.FloatTensor(surface_points_).unsqueeze(0)
    point_labels = torch.LongTensor(point_labels).unsqueeze(0).to(DEVICE)

    input_points, input_neighbors, input_pools, feat_shape = prepare_input(xyz, k=16, num_layers=3, encoder_dims = [8,32,64], sub_sampling_ratio=4, 
                                                               device=DEVICE)
    
    with torch.no_grad():
        all_logits = randla.forward(features, input_points, input_neighbors, input_pools)
        all_logits = all_logits.squeeze().detach().numpy()
        
    logits_randla_augment = scene.colorize_labels(all_logits.argmax(axis=-1))
    with open(f'../../processed/logits_folder_2/final_logits_randla_PA_{sn}.npy', 'wb') as f:
        np.save(f, logits_randla_augment)
    with open(f'../../processed/surface_points_folder_2/surface_points_randla_PA_{sn}.npy', 'wb') as f:
        np.save(f, surface_points)
    IOU = get_iou(all_logits, point_labels.detach().squeeze().numpy())
    IOU_weighted = IOU * class_weights
    #print("IOU_weighted is {}".format(IOU_weighted))
    mIOU = np.sum(IOU_weighted)
    print("mIOU given by: {}".format(mIOU))
    mIOU_list.append(mIOU)
    print(mIOU_list)

  0%|                                                                                                                                                                                       | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_d

mIOU given by: 0.4846075162424217
[0.4846075162424217]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.255456347341134
[0.4846075162424217, 0.255456347341134]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

...

mIOU given by: 0.36501413900663277
[0.4846075162424217, 0.255456347341134, 0.40655250986468694, 0.2914320068832024, 0.38134707124493505, 0.28860019671442033, 0.19555159018203142, 0.515550696790379, 0.4989175728487292, 0.42427756612955236, 0.22693985633697364, 0.5865107821643827, 0.27873171451915196, 0.4933933202146396, 0.31064573386794014, 0.31381475018319255, 0.26102244129174407, 0.7176864999981254, 0.33333968257916924, 0.5675550054291562, 0.37420679611859553, 0.45611167495040184, 0.34609760481759294, 0.34365588563021704, 0.5892706134448521, 0.25291029680399724, 0.6782325924353184, 0.30996191844035564, 0.5857200388233089, 0.6288938056985944, 0.2882286696281089, 0.21477510148995757, 0.25346003031746833, 0.5284508958892907, 0.2894926889687562, 0.3096090343192733, 0.350345046131364, 0.3524337003066292, 0.33343291712329987, 0.5143123832229106, 0.30515074062700986, 0.41481478444237063, 0.31530147678186116, 0.5467206544182754, 0.38327669114963125, 0.5778440233363761, 0.32485489847682947, 0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.4287786443736547
[0.4846075162424217, 0.255456347341134, 0.40655250986468694, 0.2914320068832024, 0.38134707124493505, 0.28860019671442033, 0.19555159018203142, 0.515550696790379, 0.4989175728487292, 0.42427756612955236, 0.22693985633697364, 0.5865107821643827, 0.27873171451915196, 0.4933933202146396, 0.31064573386794014, 0.31381475018319255, 0.26102244129174407, 0.7176864999981254, 0.33333968257916924, 0.5675550054291562, 0.37420679611859553, 0.45611167495040184, 0.34609760481759294, 0.34365588563021704, 0.5892706134448521, 0.25291029680399724, 0.6782325924353184, 0.30996191844035564, 0.5857200388233089, 0.6288938056985944, 0.2882286696281089, 0.21477510148995757, 0.25346003031746833, 0.5284508958892907, 0.2894926889687562, 0.3096090343192733, 0.350345046131364, 0.3524337003066292, 0.33343291712329987, 0.5143123832229106, 0.30515074062700986, 0.41481478444237063, 0.31530147678186116, 0.5467206544182754, 0.38327669114963125, 0.5778440233363761, 0.32485489847682947, 0.2

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.395933604047641
[0.4846075162424217, 0.255456347341134, 0.40655250986468694, 0.2914320068832024, 0.38134707124493505, 0.28860019671442033, 0.19555159018203142, 0.515550696790379, 0.4989175728487292, 0.42427756612955236, 0.22693985633697364, 0.5865107821643827, 0.27873171451915196, 0.4933933202146396, 0.31064573386794014, 0.31381475018319255, 0.26102244129174407, 0.7176864999981254, 0.33333968257916924, 0.5675550054291562, 0.37420679611859553, 0.45611167495040184, 0.34609760481759294, 0.34365588563021704, 0.5892706134448521, 0.25291029680399724, 0.6782325924353184, 0.30996191844035564, 0.5857200388233089, 0.6288938056985944, 0.2882286696281089, 0.21477510148995757, 0.25346003031746833, 0.5284508958892907, 0.2894926889687562, 0.3096090343192733, 0.350345046131364, 0.3524337003066292, 0.33343291712329987, 0.5143123832229106, 0.30515074062700986, 0.41481478444237063, 0.31530147678186116, 0.5467206544182754, 0.38327669114963125, 0.5778440233363761, 0.32485489847682947, 0.27

In [ ]:
from statistics import mean
mean(mIOU_list)
max(mIOU_list)
min(mIOU_list)
mIOU_list_RandLA_with_PA = mIOU_list
list_of_better_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if i > j ]
list_of_worse_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if j -i > 0.1 ]
mIOU_list_RandLA_with_PA.index(0.6627236761349584)
mIOU_list_RandLA[16]
test_scenes[16]
print(mIOU_list_RandLA)
print(mIOU_list_RandLA_with_PA)


In [1]:
mIOU_list = [0.4846075162424217, 0.255456347341134, 0.40655250986468694, 0.2914320068832024, 0.38134707124493505, 0.28860019671442033, 0.19555159018203142, 0.515550696790379, 0.4989175728487292, 0.42427756612955236, 0.22693985633697364, 0.5865107821643827, 0.27873171451915196, 0.4933933202146396, 0.31064573386794014, 0.31381475018319255, 0.26102244129174407, 0.7176864999981254, 0.33333968257916924, 0.5675550054291562, 0.37420679611859553, 0.45611167495040184, 0.34609760481759294, 0.34365588563021704, 0.5892706134448521, 0.25291029680399724, 0.6782325924353184, 0.30996191844035564, 0.5857200388233089, 0.6288938056985944, 0.2882286696281089, 0.21477510148995757, 0.25346003031746833, 0.5284508958892907, 0.2894926889687562, 0.3096090343192733, 0.350345046131364, 0.3524337003066292, 0.33343291712329987, 0.5143123832229106, 0.30515074062700986, 0.41481478444237063, 0.31530147678186116, 0.5467206544182754, 0.38327669114963125, 0.5778440233363761, 0.32485489847682947, 0.27005094020416365, 0.38116169310805703, 0.346001988710808, 0.6045833330084354, 0.3242577776131843, 0.24263565640595, 0.43262769701663706, 0.609380012155975, 0.5605421724173624, 0.4966394776779144, 0.46046072924674564, 0.33930249818745867, 0.32500949721604916, 0.31246091029629236, 0.5045039110297619, 0.2682751239271398, 0.7668044571077273, 0.49328090337972674, 0.24282920737111796, 0.31922335033795257, 0.24960390605760857, 0.4690446182798021, 0.3015410733939889, 0.2694347145843815, 0.418057156079899, 0.5055901147952595, 0.6827349246517502, 0.5026662709027534, 0.5592745127375499, 0.19688282908097213, 0.5695084559131941, 0.2968361729415379, 0.5535857416055341, 0.2948841006842225, 0.3265257084973807, 0.3613743247944543, 0.3624242279742678, 0.3008084075564418, 0.3348122429435048, 0.19251821730837118, 0.29905192854870044, 0.5077502785317372, 0.6460830604177084, 0.21493792719176513, 0.28762786636162946, 0.4953134324450901, 0.38435670974603736, 0.2269986960242283, 0.6338375069303666, 0.32212074255176315, 0.36501413900663277, 0.4287786443736547, 0.395933604047641]

In [4]:
from statistics import mean
mean(mIOU_list)

0.399554414179969

In [2]:
max(mIOU_list)

0.7668044571077273

In [3]:
min(mIOU_list)

0.19251821730837118

In [5]:
mIOU_list_RandLA_with_PA = mIOU_list

In [6]:
mIOU_list_RandLA = [0.4800828259274245, 0.3141256294622326, 0.43314472897323886, 0.3108838838691709, 0.4217744661366317, 0.30750138626132656, 0.23392175229936546, 0.5477260303527688, 0.4806466370151126, 0.4273892546729509, 0.23403418107045632, 0.622984223580449, 0.28395898477729614, 0.5053798888534788, 0.36854260904913283, 0.23487570158381382, 0.2776467227756309, 0.6755961181605183, 0.37903712673606926, 0.5504114010316731, 0.37429026290122175, 0.496385323357798, 0.39196129947896174, 0.37924939647851275, 0.5853269475386425, 0.255262318179988, 0.6848665717060205, 0.30138798440552894, 0.5921036056828485, 0.6546765959275835, 0.27826561055984433, 0.22863031747535176, 0.27819431972682734, 0.5282928933268631, 0.29809156976279433, 0.2563069268808263, 0.41665888131670775, 0.4455425174190389, 0.34989910732345086, 0.5253094595573349, 0.2890501560457343, 0.4635251003659016, 0.24949786498487941, 0.3703191870359074, 0.3865878299278053, 0.5965633281022387, 0.29854842588276786, 0.36455184643516014, 0.4672057744440935, 0.28608381392909565, 0.660474617707548, 0.36995875605510714, 0.21945322556332392, 0.4129921812023513, 0.6380386723577989, 0.4061284989658414, 0.4925925715161546, 0.510103539110759, 0.3514833816962437, 0.33528637051815435, 0.33819533628154197, 0.5021063505252817, 0.2877435568473731, 0.5891480131841499, 0.5533526939783981, 0.34352733370154254, 0.3384086923052993, 0.22271800487470828, 0.5073600516432464, 0.33488550937204525, 0.31599287733920467, 0.4427037114966608, 0.4858685740850179, 0.7067245979891372, 0.4690826399445896, 0.6355670688173295, 0.18795980837983148, 0.6419685226000515, 0.30466211400702536, 0.6197089997148911, 0.2722936595196873, 0.2838553068763355, 0.3647035686781857, 0.32497194534968854, 0.2999724561382861, 0.3994000120406234, 0.21950022660212365, 0.3174245182795108, 0.4636270492510911, 0.4673374997438176, 0.3154642743049199, 0.29810931767210774, 0.569316053154611, 0.40090543301875153, 0.2193442452115066, 0.6412005253106188, 0.33164283054346555, 0.41097741115303804, 0.4041420810206367, 0.4566871219049027]

In [115]:
print(mIOU_list_RandLA_with_PA)

[0.4846075162424217, 0.255456347341134, 0.40655250986468694, 0.2914320068832024, 0.38134707124493505, 0.28860019671442033, 0.19555159018203142, 0.515550696790379, 0.4989175728487292, 0.42427756612955236, 0.22693985633697364, 0.5865107821643827, 0.27873171451915196, 0.4933933202146396, 0.31064573386794014, 0.31381475018319255, 0.26102244129174407, 0.7176864999981254, 0.33333968257916924, 0.5675550054291562, 0.37420679611859553, 0.45611167495040184, 0.34609760481759294, 0.34365588563021704, 0.5892706134448521, 0.25291029680399724, 0.6782325924353184, 0.30996191844035564, 0.5857200388233089, 0.6288938056985944, 0.2882286696281089, 0.21477510148995757, 0.25346003031746833, 0.5284508958892907, 0.2894926889687562, 0.3096090343192733, 0.350345046131364, 0.3524337003066292, 0.33343291712329987, 0.5143123832229106, 0.30515074062700986, 0.41481478444237063, 0.31530147678186116, 0.5467206544182754, 0.38327669114963125, 0.5778440233363761, 0.32485489847682947, 0.27005094020416365, 0.38116169310805

In [116]:
print(mIOU_list_RandLA)

[0.4800828259274245, 0.3141256294622326, 0.43314472897323886, 0.3108838838691709, 0.4217744661366317, 0.30750138626132656, 0.23392175229936546, 0.5477260303527688, 0.4806466370151126, 0.4273892546729509, 0.23403418107045632, 0.622984223580449, 0.28395898477729614, 0.5053798888534788, 0.36854260904913283, 0.23487570158381382, 0.2776467227756309, 0.6755961181605183, 0.37903712673606926, 0.5504114010316731, 0.37429026290122175, 0.496385323357798, 0.39196129947896174, 0.37924939647851275, 0.5853269475386425, 0.255262318179988, 0.6848665717060205, 0.30138798440552894, 0.5921036056828485, 0.6546765959275835, 0.27826561055984433, 0.22863031747535176, 0.27819431972682734, 0.5282928933268631, 0.29809156976279433, 0.2563069268808263, 0.41665888131670775, 0.4455425174190389, 0.34989910732345086, 0.5253094595573349, 0.2890501560457343, 0.4635251003659016, 0.24949786498487941, 0.3703191870359074, 0.3865878299278053, 0.5965633281022387, 0.29854842588276786, 0.36455184643516014, 0.4672057744440935, 0

In [7]:
list_of_better_performing_scenes = [i for i, j in zip(mIOU_list_RandLA_with_PA, mIOU_list_RandLA) if i > j ]

In [8]:
len(list_of_better_performing_scenes)

33

In [9]:
mIOU_list_RandLA_with_PA.index(0.7668044571077273)

63

In [120]:
mIOU_list_RandLA[63]

0.5891480131841499

In [ ]:
test_scenes_2[16]

In [113]:
list_of_better_performing_scenes

[0.4846075162424217,
 0.4989175728487292,
 0.31381475018319255,
 0.7176864999981254,
 0.5675550054291562,
 0.5892706134448521,
 0.30996191844035564,
 0.2882286696281089,
 0.5284508958892907,
 0.3096090343192733,
 0.30515074062700986,
 0.31530147678186116,
 0.5467206544182754,
 0.32485489847682947,
 0.346001988710808,
 0.24263565640595,
 0.43262769701663706,
 0.5605421724173624,
 0.4966394776779144,
 0.5045039110297619,
 0.7668044571077273,
 0.24960390605760857,
 0.5055901147952595,
 0.5026662709027534,
 0.19688282908097213,
 0.2948841006842225,
 0.3265257084973807,
 0.3624242279742678,
 0.3008084075564418,
 0.5077502785317372,
 0.6460830604177084,
 0.2269986960242283,
 0.4287786443736547]